### **Paradigmas de aprendizaje (Supervisado, Few‑shot, No Supervisado, Autosupervisado/Contrastivo, Refuerzo)**





In [ ]:
# Configuración
import os, math, random
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, adjusted_rand_score
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

SEED = 7
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)




### **Dataset: Digits (8×8)**
Usaremos `sklearn.datasets.load_digits()` para evitar descargas. Es pequeño, perfecto para demos rápidas.


In [ ]:
digits = load_digits()
X = digits.data.astype(np.float32)          # (n, 64)
y = digits.target.astype(np.int64)          # (n,)
images = digits.images.astype(np.float32)   # (n, 8, 8)

# Normalización a [0,1] (los dígitos vienen típicamente en [0,16])
X01 = X / 16.0
images01 = images / 16.0

print("X01:", X01.shape, "y:", y.shape)

# Visualiza 12 ejemplos
idx = np.random.choice(len(X01), size=12, replace=False)
fig, axes = plt.subplots(3, 4, figsize=(6, 4))
for ax, i in zip(axes.ravel(), idx):
    ax.imshow(images01[i], cmap="gray", vmin=0, vmax=1)
    ax.set_title(str(y[i]))
    ax.axis("off")
plt.tight_layout()
plt.show()


#### **2. Aprendizaje supervisado**
##### **Idea**
Aprende una función $f(x) \rightarrow y$ usando pares etiquetados $(x, y)$. En el curso, esto sirve como baseline y para discutir:
- particionado train/test,
- métricas (accuracy, precision/recall),
- sobreajuste y regularización.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X01, y, test_size=0.25, random_state=SEED, stratify=y
)

# Pipeline típico: escalado -> modelo
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(max_iter=3000))
])

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred, digits=3))

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title("Matriz de confusión (LogReg)")
plt.show()


#### **Mini‑experimento: ¿qué pasa si tengo pocos datos etiquetados?**
Esto conecta con el *few‑shot como régimen de pocos datos* (pero **no** es lo mismo que meta‑learning).


In [ ]:
def train_with_label_budget(label_budget_per_class: int):
    # Mantén solo K ejemplos por clase en entrenamiento
    keep = []
    for c in np.unique(y_train):
        idx_c = np.where(y_train == c)[0]
        np.random.shuffle(idx_c)
        keep.extend(idx_c[:label_budget_per_class])
    keep = np.array(keep)

    Xb, yb = X_train[keep], y_train[keep]

    model = Pipeline([
        ("scaler", StandardScaler()),
        ("lr", LogisticRegression(max_iter=3000))
    ])
    model.fit(Xb, yb)
    acc = model.score(X_test, y_test)
    return acc

for k in [1, 2, 5, 10, 30, 100]:
    acc = train_with_label_budget(k)
    print(f"K={k:3d} labels/clase  ->  accuracy={acc:.3f}")


#### **3. Few‑shot: N‑way, K‑shot (evaluación episódica)**
##### **Definiciones operativas**
- **N‑way**: número de clases en el episodio (por ejemplo, 5‑way).
- **K‑shot**: número de ejemplos etiquetados por clase en el *support set* (por ejemplo, 1‑shot).
- *Query set*: ejemplos a clasificar, sin etiquetas.

Aquí implementamos un evaluador tipo **Prototypical Networks** (prototipos = media de embeddings del support).

Para mostrar el valor del pretraining, compararemos embeddings:
1) **Baseline**: píxeles
2) **Autoencoder** (autosupervisado)
3) **Contrastivo** (autosupervisado)



**Prototypical Networks**

Un **evaluador tipo Prototypical Networks** es una forma de hacer *clasificación few-shot* donde, en vez de entrenar un clasificador completo desde cero para cada tarea, construyes un "representante" (**prototipo**) por clase usando los pocos ejemplos disponibles.

**Support set (conjunto de soporte)**

En un episodio few-shot divides los datos en dos partes:

* **Support set**: los ejemplos **etiquetados** que te "dan" para aprender en ese episodio.
 - Ejemplo: en **5-way 1-shot**, el support set tiene **5 clases × 1 ejemplo por clase = 5 imágenes** con etiqueta.
* **Query set**: ejemplos **a clasificar** (normalmente sin etiqueta para el modelo; tú sí la tienes para evaluar).
 - Ejemplo: 5-way 1-shot con 10 queries por clase -> 50 imágenes para probar.

#### **¿Qué es un prototipo?**

Primero pasas cada ejemplo por un **encoder** que lo convierte en un **embedding** (vector) en un espacio donde "cosas parecidas quedan cerca".

Para cada clase (c), tomas los embeddings de sus ejemplos en el support set y calculas el **promedio**:

$$
\mathbf{p}*c = \frac{1}{K}\sum*{i=1}^{K} \mathbf{z}_i^{(c)}
$$

* $\mathbf{z}_i^{(c)}$: embedding del i-ésimo ejemplo del support de la clase (c)
* $K$: número de *shots* por clase (K-shot)
* $\mathbf{p}_c$: **prototipo** (centro) de la clase (c)

Intuición: el prototipo es como el **centroide** de la clase en el espacio de embeddings.

#### **¿Cómo clasifica el evaluador?**

Para cada ejemplo del **query set**:

1. Calculas su embedding $\mathbf{z}_q$.
2. Mides distancia a cada prototipo (típicamente **euclídea** o **coseno**):
   $$
   \hat{y} = \arg\min_c ; d(\mathbf{z}_q,\mathbf{p}_c)
   $$
3. Predices la clase del prototipo más cercano.



In [ ]:
@torch.no_grad()
def prototypical_episode(embeddings, labels, n_way=5, k_shot=1, q_query=10):
    """
    embeddings: (n, d) torch tensor
    labels: (n,) torch tensor long
    """
    classes = labels.unique().cpu().numpy().tolist()
    chosen = random.sample(classes, n_way)

    support_idx = []
    query_idx = []

    for c in chosen:
        idx_c = (labels == c).nonzero(as_tuple=False).view(-1).cpu().numpy()
        np.random.shuffle(idx_c)
        support_idx.extend(idx_c[:k_shot])
        query_idx.extend(idx_c[k_shot:k_shot + q_query])

    support_idx = torch.tensor(support_idx, dtype=torch.long, device=embeddings.device)
    query_idx   = torch.tensor(query_idx, dtype=torch.long, device=embeddings.device)

    E_s = embeddings[support_idx]  # (n_way*k, d)
    y_s = labels[support_idx]      # (n_way*k,)
    E_q = embeddings[query_idx]
    y_q = labels[query_idx]

    # prototipos: media por clase
    protos = []
    proto_labels = []
    for c in chosen:
        mask = (y_s == c)
        protos.append(E_s[mask].mean(dim=0))
        proto_labels.append(int(c))
    protos = torch.stack(protos, dim=0)  # (n_way, d)

    # distancia euclídea al prototipo
    dists = ((E_q.unsqueeze(1) - protos.unsqueeze(0))**2).sum(dim=-1)
    pred = dists.argmin(dim=1)  # índice en [0, n_way)
    pred_labels = torch.tensor([proto_labels[i] for i in pred.cpu().numpy()], device=embeddings.device)

    acc = (pred_labels == y_q).float().mean().item()
    return acc

@torch.no_grad()
def evaluate_fewshot(embeddings, labels, n_way=5, k_shot=1, q_query=10, episodes=200):
    accs = []
    for _ in range(episodes):
        accs.append(prototypical_episode(embeddings, labels, n_way, k_shot, q_query))
    return float(np.mean(accs)), float(np.std(accs))


##### **3.1 Baseline: embeddings = píxeles normalizados**
No es un modelo, es una referencia mínima.


In [ ]:
E_pixels = torch.tensor(X01, device=device)  # (n, 64)
Y_t = torch.tensor(y, device=device)

for (n_way, k_shot) in [(5,1), (5,5), (10,1), (10,5)]:
    mean_acc, std_acc = evaluate_fewshot(E_pixels, Y_t, n_way=n_way, k_shot=k_shot, episodes=200)
    print(f"[pixels] {n_way}-way {k_shot}-shot  acc={mean_acc:.3f} ± {std_acc:.3f}")


#### **4. No supervisado: estructura, segmentos y anomalías**
##### **4.1 Estructura: clustering (KMeans)**
Buscamos agrupamientos sin usar etiquetas. Luego (solo para la demo) comparamos con etiquetas verdaderas usando **ARI**.


In [ ]:
k = 10
kmeans = KMeans(n_clusters=k, n_init=20, random_state=SEED)
clusters = kmeans.fit_predict(X01)

ari = adjusted_rand_score(y, clusters)
print("ARI (KMeans vs etiquetas reales):", round(ari, 3))


##### **4.2 Segmentos: segmentación simple de una imagen (KMeans en pixeles)**

Ejemplo minimalista: separa *fondo vs trazo* en un dígito 8×8.


In [ ]:
i = int(np.random.choice(len(images01)))
img = images01[i].copy()
pixels = img.reshape(-1, 1)  # (64,1)

seg = KMeans(n_clusters=2, n_init=10, random_state=SEED).fit_predict(pixels)
seg_img = seg.reshape(8, 8)

# Asegura que el "trazo" sea 1 (por intensidad media)
if img[seg_img==0].mean() > img[seg_img==1].mean():
    seg_img = 1 - seg_img

fig, axes = plt.subplots(1, 3, figsize=(7, 2.5))
axes[0].imshow(img, cmap="gray", vmin=0, vmax=1); axes[0].set_title(f"Original (y={y[i]})"); axes[0].axis("off")
axes[1].imshow(seg_img, cmap="gray"); axes[1].set_title("Máscara (2 clusters)"); axes[1].axis("off")
axes[2].imshow(img * seg_img, cmap="gray", vmin=0, vmax=1); axes[2].set_title("Trazo extraído"); axes[2].axis("off")
plt.tight_layout(); plt.show()


##### **4.3 Anomalías: detección con Isolation Forest**
Creamos outliers sintéticos (ruido) y vemos si el detector los puntúa como anómalos.


In [ ]:
n_out = 150
outliers = np.clip(np.random.rand(n_out, 64).astype(np.float32), 0, 1)  # ruido uniforme
X_mix = np.vstack([X01, outliers])
y_is_outlier = np.array([0]*len(X01) + [1]*n_out)  # 1 si es outlier

iso = IsolationForest(n_estimators=300, contamination=n_out/len(X_mix), random_state=SEED)
iso.fit(X_mix)

scores = -iso.decision_function(X_mix)  # mayor = más anómalo
thr = np.quantile(scores, 1 - n_out/len(X_mix))
pred_out = (scores >= thr).astype(int)

tp = int(((pred_out==1) & (y_is_outlier==1)).sum())
fp = int(((pred_out==1) & (y_is_outlier==0)).sum())
fn = int(((pred_out==0) & (y_is_outlier==1)).sum())

precision = tp / max(tp+fp, 1)
recall = tp / max(tp+fn, 1)
print(f"precision={precision:.3f}, recall={recall:.3f} (sobre outliers sintéticos)")

plt.figure(figsize=(6,3))
plt.hist(scores[y_is_outlier==0], bins=40, alpha=0.7, label="inliers (digits)")
plt.hist(scores[y_is_outlier==1], bins=40, alpha=0.7, label="outliers (ruido)")
plt.axvline(thr, linestyle="--")
plt.title("Scores de anomalía (IsolationForest)")
plt.legend()
plt.show()


#### **5. Autosupervisado: motor del pretraining (autoencoder)**
En autosupervisado, **la señal viene de los propios datos**. En un autoencoder, la tarea pretexto es **reconstruir x**.

Luego usamos el **encoder** como extractor de embeddings para:
- N‑way K‑shot,
- clasificación con pocos labels (linear probe).


In [ ]:
class AE(nn.Module):
    def __init__(self, d_in=64, d_lat=32):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(d_in, 128), nn.ReLU(),
            nn.Linear(128, d_lat)
        )
        self.decoder = nn.Sequential(
            nn.Linear(d_lat, 128), nn.ReLU(),
            nn.Linear(128, d_in), nn.Sigmoid()
        )

    def forward(self, x):
        z = self.encoder(x)
        xhat = self.decoder(z)
        return xhat, z

def train_ae(X, epochs=15, batch_size=128, lr=1e-3):
    model = AE(d_in=X.shape[1], d_lat=32).to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=lr)
    ds = TensorDataset(torch.tensor(X, dtype=torch.float32))
    dl = DataLoader(ds, batch_size=batch_size, shuffle=True)

    for ep in range(1, epochs+1):
        model.train()
        losses = []
        for (xb,) in dl:
            xb = xb.to(device)
            xhat, _ = model(xb)
            loss = F.mse_loss(xhat, xb)
            opt.zero_grad()
            loss.backward()
            opt.step()
            losses.append(loss.item())
        if ep % 5 == 0 or ep == 1:
            print(f"Epoca {ep:02d}  recon_mse={np.mean(losses):.5f}")
    return model

ae = train_ae(X01, epochs=15)


In [ ]:
@torch.no_grad()
def encode_with(model, X):
    model.eval()
    X_t = torch.tensor(X, dtype=torch.float32, device=device)
    _, z = model(X_t)
    return z

Z_ae = encode_with(ae, X01)  # (n, d_lat)

# Muestra reconstrucciones
ae.eval()
idx = np.random.choice(len(X01), size=8, replace=False)
xb = torch.tensor(X01[idx], dtype=torch.float32, device=device)

with torch.no_grad():
    xhat, _ = ae(xb)

xhat = xhat.cpu().numpy().reshape(-1, 8, 8)

fig, axes = plt.subplots(2, 8, figsize=(12, 3))
for j in range(8):
    axes[0, j].imshow(images01[idx[j]], cmap="gray", vmin=0, vmax=1); axes[0, j].axis("off")
    axes[1, j].imshow(xhat[j], cmap="gray", vmin=0, vmax=1); axes[1, j].axis("off")
axes[0,0].set_ylabel("orig", rotation=0, labelpad=25)
axes[1,0].set_ylabel("recon", rotation=0, labelpad=25)
plt.suptitle("Autoencoder: original vs reconstrucción")
plt.show()


##### **5.1 Few‑shot con embeddings del autoencoder**


In [ ]:
for (n_way, k_shot) in [(5,1), (5,5), (10,1), (10,5)]:
    mean_acc, std_acc = evaluate_fewshot(Z_ae, Y_t, n_way=n_way, k_shot=k_shot, episodes=200)
    print(f"[AE emb] {n_way}-way {k_shot}-shot  acc={mean_acc:.3f} ± {std_acc:.3f}")


##### **5.2 Clasificación supervisada con *pocos labels* + encoder congelado**
Entrenamos un clasificador lineal sobre embeddings **Z** usando solo K etiquetas por clase.


In [ ]:
Z_np = Z_ae.detach().cpu().numpy()
Z_train, Z_test, y_train2, y_test2 = train_test_split(Z_np, y, test_size=0.25, random_state=SEED, stratify=y)

def linear_probe_with_budget(Ztr, ytr, Zte, yte, label_budget_per_class: int):
    keep = []
    for c in np.unique(ytr):
        idx_c = np.where(ytr == c)[0]
        np.random.shuffle(idx_c)
        keep.extend(idx_c[:label_budget_per_class])
    keep = np.array(keep)

    model = LogisticRegression(max_iter=3000)
    model.fit(Ztr[keep], ytr[keep])
    return model.score(Zte, yte)

for k in [1, 2, 5, 10, 30, 100]:
    acc = linear_probe_with_budget(Z_train, y_train2, Z_test, y_test2, k)
    print(f"[AE+linear] K={k:3d} labels/clase -> accuracy={acc:.3f}")


#### **6. Autosupervisado: señal contrastiva (contrastive learning)**

**Autosupervisado** es aprender representaciones **sin etiquetas humanas**, creando una "señal" desde los propios datos. En Digits, no usamos la clase (0-9) para entrenar, en vez de eso, definimos una tarea pretexto: *dos versiones del mismo dígito deben parecerse en el embedding*.

Ahí entra **contrastive learning**: es un tipo de autosupervisado donde la señal es **comparar pares**. En estilo **SimCLR** con Digits:

1. Tomamos una imagen (x) (por ejemplo. un "3").
2. Generamos dos **vistas** $(v_1, v_2)$ con augmentations (ruido, masking de píxeles).
3. Pasamos ambas por el mismo encoder (f$\cdot$) y obtenemos embeddings $(z_1, z_2)$.
4. La pérdida **NT-Xent** obliga a que $z_1$ y $z_2$ estén **cerca** (par positivo: mismo ejemplo) y que se alejen de embeddings de otras imágenes del batch (pares negativos: ejemplos distintos).

Resultado: el encoder aprende un espacio donde "dígitos similares" tienden a agruparse, y luego sirve para **few-shot** o clasificación con pocos labels.



In [ ]:
class Encoder(nn.Module):
    def __init__(self, d_in=64, d_emb=32):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_in, 128), nn.ReLU(),
            nn.Linear(128, d_emb)
        )
    def forward(self, x):
        return self.net(x)

def augment_digits(x):
    """x: (batch, 64) en [0,1]"""
    noise = 0.10 * torch.randn_like(x)
    x2 = (x + noise).clamp(0, 1)
    # masking aleatorio de 10% de pixeles
    mask = (torch.rand_like(x2) < 0.10).float()
    x2 = x2 * (1 - mask)
    return x2

def nt_xent(z1, z2, tau=0.2):
    z1 = F.normalize(z1, dim=1)
    z2 = F.normalize(z2, dim=1)
    B = z1.size(0)

    z = torch.cat([z1, z2], dim=0)  # (2B, d)
    sim = torch.matmul(z, z.T) / tau  # (2B, 2B)

    diag = torch.eye(2*B, device=z.device).bool()
    sim = sim.masked_fill(diag, -1e9)

    # positivos: (i, i+B) y (i+B, i)
    pos = torch.cat([torch.arange(B, 2*B), torch.arange(0, B)]).to(z.device)
    return F.cross_entropy(sim, pos)

def train_contrastive(X, epochs=25, batch_size=256, lr=2e-3):
    enc = Encoder(d_in=X.shape[1], d_emb=32).to(device)
    opt = torch.optim.AdamW(enc.parameters(), lr=lr)

    ds = TensorDataset(torch.tensor(X, dtype=torch.float32))
    dl = DataLoader(ds, batch_size=batch_size, shuffle=True, drop_last=True)

    for ep in range(1, epochs+1):
        enc.train()
        losses = []
        for (xb,) in dl:
            xb = xb.to(device)
            v1 = augment_digits(xb)
            v2 = augment_digits(xb)

            z1 = enc(v1)
            z2 = enc(v2)

            loss = nt_xent(z1, z2, tau=0.2)
            opt.zero_grad()
            loss.backward()
            opt.step()
            losses.append(loss.item())
        if ep % 5 == 0 or ep == 1:
            print(f"Epoca {ep:02d}  nt_xent={np.mean(losses):.4f}")
    return enc

enc = train_contrastive(X01, epochs=25)


In [ ]:
@torch.no_grad()
def encode_enc(enc, X):
    enc.eval()
    X_t = torch.tensor(X, dtype=torch.float32, device=device)
    return enc(X_t)

Z_con = encode_enc(enc, X01)

for (n_way, k_shot) in [(5,1), (5,5), (10,1), (10,5)]:
    mean_acc, std_acc = evaluate_fewshot(Z_con, Y_t, n_way=n_way, k_shot=k_shot, episodes=200)
    print(f"[Contrast emb] {n_way}-way {k_shot}-shot  acc={mean_acc:.3f} ± {std_acc:.3f}")


#### **7. Aprendizaje por refuerzo: GridWorld + Q‑learning**
Implementación desde cero para discutir MDP, política, valor y aprendizaje por interacción.


In [ ]:
class GridWorld:
    def __init__(self, n=5):
        self.n = n
        self.start = (0, 0)
        self.goal = (n-1, n-1)
        self.traps = {(1, 3), (2, 1), (3, 3)}
        self.walls = {(1, 1), (2, 3)}  # no se puede entrar
        self.reset()

    def reset(self):
        self.s = self.start
        return self.state_id(self.s)

    def state_id(self, s):
        return s[0] * self.n + s[1]

    def step(self, a):
        r, c = self.s
        drdc = {0:(-1,0), 1:(0,1), 2:(1,0), 3:(0,-1)}[a]
        nr, nc = r + drdc[0], c + drdc[1]

        nr = max(0, min(self.n-1, nr))
        nc = max(0, min(self.n-1, nc))

        if (nr, nc) in self.walls:
            nr, nc = r, c

        self.s = (nr, nc)

        reward = -0.01
        done = False
        if self.s in self.traps:
            reward = -1.0
            done = True
        elif self.s == self.goal:
            reward = 1.0
            done = True

        return self.state_id(self.s), reward, done

env = GridWorld(n=5)
nS = env.n * env.n
nA = 4


In [ ]:
def q_learning(env, episodes=4000, alpha=0.15, gamma=0.98, eps0=1.0, eps_min=0.05, eps_decay=0.999):
    Q = np.zeros((nS, nA), dtype=np.float32)
    rewards = []

    eps = eps0
    for ep in range(episodes):
        s = env.reset()
        ep_ret = 0.0

        for t in range(200):
            if np.random.rand() < eps:
                a = np.random.randint(nA)
            else:
                a = int(np.argmax(Q[s]))

            s2, r, done = env.step(a)
            ep_ret += r

            Q[s, a] = Q[s, a] + alpha * (r + gamma * np.max(Q[s2]) - Q[s, a])
            s = s2

            if done:
                break

        rewards.append(ep_ret)
        eps = max(eps_min, eps * eps_decay)

    return Q, np.array(rewards)

Q, rets = q_learning(env)

window = 100
mov = np.convolve(rets, np.ones(window)/window, mode="valid")

plt.figure(figsize=(6,3))
plt.plot(mov)
plt.title(f"Q-learning: retorno promedio móvil (ventana={window})")
plt.xlabel("episodio")
plt.ylabel("retorno")
plt.show()


In [ ]:
def render_policy(env, Q):
    arrows = {0:"↑", 1:"→", 2:"↓", 3:"←"}
    grid = []
    for r in range(env.n):
        row = []
        for c in range(env.n):
            if (r,c) == env.start:
                row.append("S")
            elif (r,c) == env.goal:
                row.append("G")
            elif (r,c) in env.walls:
                row.append("█")
            elif (r,c) in env.traps:
                row.append("X")
            else:
                sid = env.state_id((r,c))
                row.append(arrows[int(np.argmax(Q[sid]))])
        grid.append(row)
    return grid

pol = render_policy(env, Q)
for row in pol:
    print(" ".join(row))


#### **8. Ejercicios**

##### **Aprendizaje Supervisado**

1. **MLPClassifier vs LogReg (trade-off precisión/tiempo)**

   * Reemplaza `LogisticRegression` por `sklearn.neural_network.MLPClassifier`.
   * Prueba 3 configuraciones: `hidden_layer_sizes=(32,)`, `(64,)`, `(128,64)` y activa `early_stopping=True`.
   * Reporta: `accuracy`, tiempo de entrenamiento (usa `time.perf_counter()`), y una matriz de confusión.
   * Discute cuándo conviene un MLP "pequeño" frente a un modelo lineal en Digits.

2. **Calibración de probabilidades (reliability)**

   * Compara calibración de LogReg y MLP con `CalibratedClassifierCV` (métodos `sigmoid` y `isotonic`).
   * Mide **Brier score** y dibuja **reliability diagram** (curva de calibración).
   * Identifica si el modelo es *overconfident* (probabilidades altas incorrectas) o *underconfident*.

##### **Few-shot (N-way/K-shot con prototipos)**

3. **Curva K-shot (con barras de error)**

   * Para `N ∈ {5, 10}` y `K ∈ {1,2,5,10}`, ejecuta `evaluate_fewshot(..., episodes=500)` y guarda `mean±std`.
   * Grafica 4 curvas: (5-way y 10-way) × (embeddings píxeles vs embeddings AE vs embeddings contrastivos).
   * Observa qué representación "sube más rápido" al aumentar K.

4. **Cosine distance + normalización (ablation)**

   * Modifica el episodio prototípico para usar **cosine distance**: normaliza `E_q` y `protos` con `F.normalize`.
   * Compara contra euclídea para las 4 combinaciones (N,K).
   * Discute por qué cosine suele ayudar cuando los embeddings están aprendidos y normalizados.

##### **Aprendizaje no supervisado**

5. **Clustering: sensibilidad a k + estabilidad**

   * Evalúa `k ∈ {8,9,10,11,12}` con `n_init` alto.
   * Reporta: inercia (`kmeans.inertia_`) y (solo para la demo) ARI.
   * Repite con 5 semillas distintas y mide varianza: ¿qué tan estable es el agrupamiento?

6. **Anomalías: outliers "difíciles" y degradación**

   * Reemplaza outliers de "ruido puro" por perturbaciones *cercanas* a Digits:

     (a) masking fuerte de píxeles, (b) ruido gaussiano suave, (c) inversión parcial (1−x) en regiones.
   * Mantén la misma tasa de contaminación y reporta precisión/recall de detección.
   * Explica qué tipo de outlier engaña más al detector y por qué.

##### **Autosupervisado**

7. **Autoencoder: capacidad latente y utilidad real**

   * Entrena AE con `d_lat ∈ {8,16,32,64}`.
   * Para cada uno: (a) MSE de reconstrucción, (b) few-shot (5-way 1-shot y 10-way 1-shot), (c) linear probe con K=5 labels/clase.

8. **Contrastivo: temperatura $\tau$ y diseño de augmentations (ablation serio)**

   * Entrena 3 modelos con `tau ∈ {0.1,0.2,0.5}` manteniendo todo lo demás fijo.
   * Luego fija τ y cambia augmentations:

      * solo ruido, solo masking, ruido+masking.
   * Evalúa few-shot y compara. Conclusión esperada: *augmentations definen qué invariancias aprende el embedding*.

##### **Aprendizaje por Refuerzo**

9. **Shaping de recompensas y política emergente**

   * Cambia la penalidad por paso `{-0.00, -0.01, -0.05}` y añade (opcional) penalidad por chocar pared.
   * Para cada setting: curva de retorno promedio móvil y política final impresa.
   * Interpreta: ¿se vuelve más "agresiva" buscando la meta? ¿evita trampas antes?

10. **Epsilon-greedy: decaimiento lineal vs exponencial**

    * Implementa ε lineal: de 1.0 a 0.05 en, por ejemplo, 60% de episodios; luego constante.
    * Compara contra el decaimiento exponencial actual (tu baseline).
    * Reporta: velocidad de convergencia (episodio donde el retorno se estabiliza) y robustez (varianza entre 3 semillas).


In [ ]:
## Tus respuestas